In [10]:
import requests
import json
import time
import re
import datetime
import yaml

In [2]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']

In [3]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

In [4]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

In [5]:
params = (
    ('price', 'BA'),
    ('from', '2016-10-17T15:00:00.000000000Z'),
    ('granularity', 'D'),
)


In [6]:
response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles', headers=headers, params=params).json()

In [11]:
response.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [12]:
response['candles'][0]

{'complete': True,
 'volume': 14634,
 'time': '2016-10-16T21:00:00.000000000Z',
 'bid': {'o': '1.09671', 'h': '1.10073', 'l': '1.09632', 'c': '1.09986'},
 'ask': {'o': '1.09712', 'h': '1.10087', 'l': '1.09645', 'c': '1.10004'}}